In [1]:
import PIL
import fitz
import pandas as pd 
from PIL import Image
import io
import re

In [3]:
doc = fitz.open('C:/Users/Escanor/Desktop/Iron HAck/Projetos/Projeto obrigatorio/Pathfinder/Data/Pathfinder_Bestiary_2.pdf') # abrindo o PDF


obs:
         
Pagina especifica com problemas:  pg 11(resolvido)  - 
    
problema - nome do monstro em 1 pagina e image na prox pagina. Colocar algo que caso encontre apenas o nome e não a imagem, pegar a imagem da proxima pagina.  - Resolvido
    
problema - pagina que não tem nada como por exemplo 9. colocar algo que caso as 2 funcoes não funcionem ignorar o resultado.  - Resolvido

problema - 2 monstros na mesma pagina pg 16  - Resolvido

problema - monstro com subcategoria como dragões, pg 92 e 93 -  Resolvido

# Tratamenteo para pegar imagem

In [45]:
def req_image (argumento):
    '''recebe um int(numero da pagina), substitue este valor e extrai a imagem da pg em questão'''
    page_image = doc[argumento] # salvando a pagina [numero da pagina] como page
    
    list_image = page_image.get_images() #  lista de imagens da pagina escolhida

    print(list_image)
    
    for item in list_image: 
            if item[3] >100 and item[3] < 1603 and item[2] >150 and item[8] == 'DCTDecode' :   # escolha da imagem pelo tamanho, quarto parametro acima de 100 e diferente de 1603, oitvo parametro da imagem igual a DCTDecode 
                img_creature = item
            else:
                ''
    imagem_isolada = doc.extract_image(img_creature[0])['image']
    
    return imagem_isolada
    
    
    

In [43]:
req_image(160)
x

UnboundLocalError: local variable 'img_creature' referenced before assignment

In [63]:
page_image = doc[160] # salvando a pagina [numero da pagina] como page

list_image = page_image.get_images() #  lista de imagens da pagina escolhida

print(list_image)

for item in list_image: 
        if item[3] >100 and item[3] < 1603 and item[2] >150 and item[8] == 'DCTDecode' :   # escolha da imagem pelo tamanho, quarto parametro acima de 100 e diferente de 1603, oitvo parametro da imagem igual a DCTDecode 
            img_creature = item
        else:
            ''
imagem_isolada = doc.extract_image(img_creature[0])['image']

return imagem_isolada




[(1773, 3411, 54, 53, 8, 'ICCBased', '', 'Im5', 'FlateDecode'), (2431, 3880, 686, 1063, 8, 'ICCBased', '', 'Im1', 'FlateDecode'), (1645, 3270, 226, 63, 8, 'ICCBased', '', 'Im2', 'FlateDecode'), (1697, 3358, 54, 53, 8, 'ICCBased', '', 'Im4', 'FlateDecode'), (1834, 3464, 54, 53, 8, 'ICCBased', '', 'Im3', 'FlateDecode'), (1629, 0, 1261, 1636, 8, 'ICCBased', '', 'Im0', 'DCTDecode')]


NameError: name 'img_creature' is not defined

In [65]:
page_image.get_images()

Imagem_pronta = PIL.Image.open(io.BytesIO(page_image.get_images(2)))  # para testar se a função está funcionando.
Imagem_pronta

TypeError: a bytes-like object is required, not 'list'

# Tratamento para pegar nome do monstro

In [37]:
def req_text(argument):
    
    '''Função recebe um int(numero da pagina) e retorna o nome limpo que está com a segunda coordenada de 135.28799438476562  '''
    
    page_text = doc[argument]  # salvando a pagina [numero da pagina] como page_text 

    words = page_text.get_text("words")  # extraindo todas as palavras da pagina 1
    
    lista_nomes = []
    
    for i in words:   # for para pegar o nomes do monstro na pagina. So funciona se o nome estiver com o segundo valor de 135.28799438476562
        
        for image in i:
            if image == 135.28799438476562:
                
                x = i[4]
                
                lista_nomes.append(x+' ')
            
            else:
                ''
    
    lista_nomes_sujos =''.join(lista_nomes)  
    
    
    if re.findall('.*?\d+/?\d?',lista_nomes_sujos) != []: # if para separar os nomes quando aparece algum numero
        lista_nomes_limpa = re.findall('.*?\d+/?\d?',lista_nomes_sujos)
    
    else: # else para selecionar os nomes que não tem numero
        lista_nomes_limpa = re.findall('\w*.\w*',lista_nomes_sujos)
    
    lista_nomes_limpa2 = [n.replace('/','_') for n in lista_nomes_limpa ]    
    
    return lista_nomes_limpa2



In [29]:
# x = req_text(9)  # para testar se a função está funcionando
# x
# name_monster = str(req_text(16)[0])
# name_monster

['AkhAnA CR 12']

# funcao para salvar a imagem com o nome certo

In [38]:

def img_names_b2 (pg):
    '''Funcao recebe o numero da pagina, roda a função para pegar o nome do monstro. 
    caso o nome do monstro não seja um grupo vzio, tenta rodar a função para pegar a image. Caso a função da imagem de erro, pega a imagem da proxima pagina. e salva na pasta escolhida.
    
    Caso o nome do monstro seja vazio, não tentar pegar mais nada e retorna a menssagem 'Vazio' '''   
    

    if req_text (pg) != []: # if para filtrar quando há nome na pagina
        names = (req_text (pg))    

        for item in names:   # rodando este For para pegar cada item separado quando ha mais de 1 monstro por pagina
            item = str(item)
            name_monster = item
            
        #name_monster = str(req_text(pg)[0])  # funcao req_text para pegar o nome do monstro e tranfomando o nome em str
            try:
                    imagem_pronta = PIL.Image.open(io.BytesIO(req_image(pg)))# abrindo a imagem gerada pela funcao req_image
                
                    imagem_pronta.save('C:/Users/Escanor/Desktop/Iron HAck/Projetos/Projeto obrigatorio/Pathfinder/img_name_b2/'+name_monster+'.jpg')
                
                    print ('imagem salva com o nome de :'+name_monster)
                                              
            except:
                    imagem_pronta = PIL.Image.open(io.BytesIO(req_image(pg+1)))# abrindo a imagem gerada pela funcao req_image
                
                    imagem_pronta.save('C:/Users/Escanor/Desktop/Iron HAck/Projetos/Projeto obrigatorio/Pathfinder/img_name_b2/'+name_monster+'.jpg')
                
                    return 'imagem da proxima pagina salva com o nome de :'+ name_monster

    
    elif req_text (pg) == []: # elif para filtrar quando não ha nome da pagina
        return 'vazio'
        
    
    else: # else para ajudar caso de algum erro em algum lugar estranho. 
        return 'erro'
    
    return 'image(s) Salva'


In [39]:
for i in range (7,292):
    img_names_b2(i)
    
    
    

imagem salva com o nome de :AChAIERAI CR 5
imagem salva com o nome de :AkhAnA CR 12
imagem salva com o nome de :BythOS CR 16
imagem salva com o nome de :PARACLetuS CR 2
imagem salva com o nome de :theLetOS CR 7
imagem salva com o nome de :CetACeAL CR 15
imagem salva com o nome de :LeOnAL CR 12
imagem salva com o nome de :SILvAnShee CR 2
imagem salva com o nome de :AgAthIOn, vuLPInAL CR 6
imagem salva com o nome de :AkAtA CR 1
imagem salva com o nome de :AmoebA SwArm Cr 1
imagem salva com o nome de : AmoebA, gIAnt Cr 1
imagem salva com o nome de :AmPhISbAenA Cr 4
imagem salva com o nome de :CASSISIAn Cr 2
imagem salva com o nome de :monADIC DevA Cr 12
imagem salva com o nome de :movAnIC DevA Cr 10
imagem salva com o nome de :AnImAte DreAm Cr 8
imagem salva com o nome de :ArAneA Cr 4
imagem salva com o nome de :ShIelD ArChon Cr 10
imagem salva com o nome de :StAr ArChon Cr 19
imagem salva com o nome de :AthACh Cr 12
imagem salva com o nome de :AttIC whISPerer Cr 4
imagem salva com o nome

UnboundLocalError: local variable 'img_creature' referenced before assignment

In [31]:
#  page_text = doc[9]  # salvando a pagina [numero da pagina] como page_text 
#  words = page_text.get_text("words") 
#  
#  words